### 허깅페이스 Lanchain

#### fill-mask 등 현재 langchain과 huggingface 연동이 지원되지 않을 때 직접 pipeline 구축하여 사용

In [1]:
import torch, os
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Model and tokenizer
model_name = 'bigscience/bloomz-7b1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Question answering pipeline
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)

# Question and context
question = "Who is Son Heung Min?"
context = """
    Son Heung-min is a South Korean professional footballer who plays as a forward for Premier League club Tottenham Hotspur and captains the South Korea national team. 
    Considered one of the best forwards in the world, he is known for his explosive speed, finishing, and two-footed ability.
"""

# Answering the question
result = qa_pipeline(question=question, context=context)
print(f"Answer: {result['answer']}")

c:\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kevin\.cache\huggingface\hub\models--bigscience--bloomz-7b1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` en

KeyboardInterrupt: 

#### huggingface와 langchain pipeline 지원될 때 아래와 같이 연동하여 사용

In [2]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = config['HUGGINGFACE_API']['HUGGINGFACEHUB_API_TOKEN']

from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub

# HuggingFace Repository ID
repo_id = 'mistralai/Mistral-7B-Instruct-v0.3'
# repo_id = 'bigscience/bloomz-7b1'

# 질의내용
question = "Who is Son Heung Min?"

# 템플릿
template = """Question: {question}

Answer: """

# 프롬프트 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=["question"])

# HuggingFaceHub 객체 생성
llm = HuggingFaceHub(
    repo_id=repo_id, 
    model_kwargs={"temperature": 0.2, 
                  "max_length": 128}
)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 실행
print(llm_chain.run(question=question))


c:\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
c:\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Question: Who is Son Heung Min?

Answer:  Son Heung-min is a South Korean professional footballer who plays as a winger for English Premier League club Tottenham Hotspur and the South Korea national team. He is known for his pace, dribbling skills, and ability to score goals. Son joined Tottenham from Bayer Leverkusen in 2015 and has since become a key player for the club, helping them to reach the Champions League final in 2019. He


In [3]:
# 질의내용
# question = "Who is Son Heung Min?"
question = "윤석열이 누구지?"

In [7]:
# HuggingFace Repository ID
# repo_id = 'mistralai/Mistral-7B-Instruct-v0.3'          # Answer: 윤석열은 한국의 유명한 과학자입니다. 그는 1933년 생겼으며, 1986년 세계과학자 협회에서 과학 평화 상을 수상했습니다. 그는 화학과 물리학에 대한 연구를 많이 
# repo_id = 'Bllossom/llama-3-Korean-Bllossom-70B'      # 에러남. 
repo_id = 'yanolja/EEVE-Korean-Instruct-10.8B-v1.0'     # 에러남. 

# repo_id = 'bigscience/bloomz-7b1'


# 템플릿
template = """Question: {question}

Answer: """

# 프롬프트 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=["question"])

# HuggingFaceHub 객체 생성
llm = HuggingFaceHub(
    repo_id=repo_id, 
    model_kwargs={"temperature": 0.2, 
                  "max_length": 128}
)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 실행
print(llm_chain.run(question=question))

HfHubHTTPError:  (Request ID: ZSiwQsFPUcstQsYhcalgT)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/yanolja/EEVE-Korean-Instruct-10.8B-v1.0.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model yanolja/EEVE-Korean-Instruct-10.8B-v1.0 is too large to be loaded automatically (21GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).